In [15]:

'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 6

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
  
  
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

data_slice = 10000
x_train = x_train[:data_slice,:]
x_test = x_test[:data_slice,:]
y_test = y_test[:data_slice,:]
y_train = y_train[:data_slice,:]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape)) #64*(3*3)*1 car non RGB
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) #prend la valeur max sur un carré de 2, stride = 2 par défaut donc dimension/2 qu'en entrée
model.add(Dropout(0.25)) #25% des poids forcés à 0
model.add(Flatten()) #on prend les cartes et on les met en vecteur
model.add(Dense(128, activation='relu')) #couche fully connected de 128 neurones (chaque neurone cablé à l'ensemble des entrées)
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) #nombres de classes qu'on veut car sortie, softmax pour avoir une valeur homogène à une probabilité

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (10000, 28, 28, 1)
10000 train samples
10000 test samples
Train on 10000 samples, validate on 10000 samples
Epoch 1/6
10000/10000 [==============================] - 14s 1ms/step - loss: 0.7362 - acc: 0.7693 - val_loss: 0.3483 - val_acc: 0.8889
Epoch 2/6
10000/10000 [==============================] - 16s 2ms/step - loss: 0.2609 - acc: 0.9231 - val_loss: 0.1502 - val_acc: 0.9561
Epoch 3/6
10000/10000 [==============================] - 12s 1ms/step - loss: 0.1762 - acc: 0.9485 - val_loss: 0.0986 - val_acc: 0.9689
Epoch 4/6
10000/10000 [==============================] - 11s 1ms/step - loss: 0.1289 - acc: 0.9626 - val_loss: 0.0925 - val_acc: 0.9706
Epoch 5/6
10000/10000 [==============================] - 13s 1ms/step - loss: 0.0997 - acc: 0.9698 - val_loss: 0.0821 - val_acc: 0.9745
Epoch 6/6
10000/10000 [==============================] - 13s 1ms/step - loss: 0.0935 - acc: 0.9723 - val_loss: 0.0698 - val_acc: 0.9790
Test loss: 0.06979303298948798
Test accuracy: 0.979


Great accuracy and test loss with only 6 epochs

In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [7]:
print(y_train.shape)

(60000,)


A chaque couche conv2d, l'image est un petit peu rognée